In [3]:
import numpy as np
import pandas as pd

# --- Experiment parameters ---
n = 500          # matrix dimension
k = 50           # target rank for approximation
ls = [int(0.05*n), int(0.1*n), int(0.2*n)]  # number of columns
reps = 5         # repetitions per setting

# --- Generate matrices ---
matrices = {
    "M1_uniform_energy": make_M1(n),
    "M2_long_tail":     make_M2(n),
    "M3_diag_variation": make_M3(n),
    "M4_clustered":     make_M4(n),
    "M5_low_rank_coherent": make_M5(n)
}

# --- Sampling methods dict ---
methods = {
    "Uniform_NR": lambda K, l: Uniform_nr(K.shape[0], l, random_state=42),
    "Diagonal":   lambda K, l: Diagonal(K, l, replace=False, random_state=42),
    "ColNorm":    lambda K, l: Column_norm(K, l, replace=False, random_state=42),
    "ICL":        lambda K, l: ICL(K, l),
    "SMGA":       lambda K, l: SMGA(K, l, random_state=42),
    "KMeans":     lambda K, l: Kmeans(K, l, random_state=42)
}

# --- Run experiments ---
results = []
for name, K in matrices.items():
    # Precompute optimal low-rank approx via full SVD
    U, S, Vt = np.linalg.svd(K, full_matrices=False)
    K_opt = (U[:, :k] * S[:k]) @ Vt[:k, :]
    opt_err = np.linalg.norm(K - K_opt, 'fro')

    for method_name, sampler in methods.items():
        for l in ls:
            errs = []
            for _ in range(reps):
                idx = sampler(K, l)
                K_hat = nystrom(K, idx, k)
                err = np.linalg.norm(K - K_hat, 'fro')
                errs.append(err)
            mean_err = np.mean(errs)
            std_err = np.std(errs)
            rel_acc = (opt_err / mean_err) * 100
            results.append({
                'Matrix': name,
                'Method': method_name,
                'l/n': round(l/n, 2),
                'RelAcc (%)': rel_acc,
                'StdErr': std_err
            })

# Save to CSV
df = pd.DataFrame(results)
df.to_csv('sampling_results.csv', index=False)
print(df.pivot(index=['Matrix','l/n'], columns='Method', values='RelAcc (%)'))


NameError: name 'null' is not defined